In [1]:
import pandas as pd
import sys
sys.path.insert(0, '../../data/lib/')
import consts
from pdfexport import *

In [3]:
ACCUM_IN = '../../data/3. transformation/1_accumulations_all.csv'
FILE_OUT = '../../data/99. analyzes/excel/'

In [57]:
#Load Data
df_hcp_raw = pd.read_csv('../../data/3. transformation/6_hcp_grouped.csv')
df_hco_raw = pd.read_csv('../../data/3. transformation/6_hco_grouped.csv')

df_raw_data = pd.concat([df_hcp, df_hco])

df_raw_accumulations = pd.read_csv(ACCUM_IN)

/Users/hws/Documents/python/pharmagelder/env/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


## prepare not anonyms

In [80]:
df_hco = df_hco_raw.copy()
df_hcp_raw = df_hcp_raw.copy()

#Sum hcp
df_hcp = df_hcp.groupby(['year']).agg({'total': 'sum'})
df_hcp = df_hcp.reset_index()
df_hcp['type'] = 'hcp'

#Sum hco
df_hco = df_hco.groupby(['year']).agg({'total': 'sum'})
df_hco = df_hco.reset_index()
df_hco['type'] = 'hco'

#df_hcp_raw = df_hcp_raw.groupby(['source', 'year']).agg({'total': 'sum'})

df_hcp.head()

,year,total,type
0,2015,8537058.93,hcp
1,2016,9351485.74,hcp
2,2017,8588884.99,hcp


## prepare accumulations

In [81]:
df_accu = df_raw_accumulations.copy()

df_accu = df_accu.replace("hcp_amount", 'hcp')
df_accu = df_accu.replace("hco_amount", 'hco')

df_accu = df_accu[~df_accu['type'].isin(['hcp_count', 'hco_count'])]

#Sum hcp
df_accu = df_accu.groupby(['type', 'year']).agg({'total': 'sum'})
df_accu = df_accu.reset_index()
#df_hcp['type'] = 'hcp'

df_accu.head(10)

,type,year,total
0,hco,2015,0.00
1,hco,2016,15605938.20
2,hco,2017,8857981.42
3,hcp,2015,0.00
4,hcp,2016,4444717.52
5,hcp,2017,3891288.54
6,rnd,2015,53935203.52
7,rnd,2016,48666896.98
8,rnd,2017,59199492.49


## process

In [92]:
df_data = pd.concat([df_accu, df_hcp, df_hco], sort=False)

#group
df_data = df_data.groupby(['type', 'year']).agg({'total': 'sum'})
df_data = df_data.reset_index()

#df_data.head(10)
df_pivot = pd.pivot_table(df_data, 'total', 'year', 'type', aggfunc='sum')
df_pivot = df_pivot.apply(lambda x: round(x, 0))
df_pivot.head()

type,hco,hcp,rnd
year,,,
2015,54530379.0,8537059.0,53935204.0
2016,89638094.0,13796203.0,48666897.0
2017,91271097.0,12480174.0,59199492.0
